In [ ]:
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage

# Load the dataset
file_path = "C:\\Users\\Devils Republic\\OneDrive\\Documents\\addresses_company_missing_new.csv"
df = pd.read_csv(file_path)

# Ensure column names have no leading/trailing spaces
df.columns = df.columns.str.strip()

# Initialize LangChain LLM (Replace with your OpenAI API Key)
llm = ChatOpenAI(model_name="gpt-4", openai_api_key="YOUR_API_KEY")

def get_location(company, Street_Name, place):
    """
    Fetch the pincode from the dataset or use GPT-4 if not found.
    """
    # Convert input to lowercase for case-insensitive matching
    company, location, place = company.lower(), Street_Name.lower(), place.lower()

    # Filter dataset based on user input
    result = df[
        (df["Company Name"].str.lower() == company) & 
        (df["Street Name"].str.lower() ==  Street_Name) & 
        (df["Place"].str.lower() == place)
    ]

    # If found and not NaN, return the pincode
    if not result.empty:
        #location = result["Location"].dropna().astype(int).values  # Convert to int safely
        if len(Street_Name) > 0:
            return Street_Name[0]

    # If not found, use GPT-4 to suggest a pincode
    query = f"What is the state for {place}, {location} in India?"
    response = llm.invoke([
        SystemMessage(content="You are an AI that takes company name, street name and local place as input, your job is to find the District location and its State in India."),
        HumanMessage(content=query)
    ])
    
    return response.content  # Return AI-suggested pincode


   

In [12]:
missing_count = 0
for index, row in df[df["Location"].isna()].iterrows():
    company, Street_Name, place = row["Company Name"], row["Street Name"], row["Place"]
    location = get_location(company, Street_Name, place)
    
    # Update DataFrame with new pincode
    df.at[index, "Location"] = location
    missing_count += 1

In [34]:
updated_file_path = "locations_company_filled.csv"
df.to_csv(updated_file_path, index=False)

#print(f"✅ Filled {missing_count} missing pincodes. Updated dataset saved as: {updated_file_path}")
print(f"✅ location data saved as: {updated_file_path}")

✅ location data saved as: locations_company_filled.csv


In [30]:
# Take user input
if __name__ == "__main__":
    company = input("Enter Company Name: ")
    Street_Name = input("Enter street Name: ")
    place = input("Enter Place Name: ")


In [31]:
 # Get Pincode
location = get_location(company,Street_Name, place)
print(f"The location for {place}, {Street_Name} ({company}) is: {location}")


The location for Madhapur, Jubilee Enclave (Delloite) is: The district for Madhapur, Jubilee Enclave in India is Ranga Reddy district and the state is Telangana.
